In [10]:
# Install required libraries (usually pre-installed in Colab, but just in case)
!pip install opencv-python numpy ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [14]:
import cv2
import numpy as np
from collections import deque
from google.colab import files  # For file upload
from ultralytics import YOLO

class ObjectTracker:
    def __init__(self, max_objects=50, min_confidence=0.5):
        self.max_objects = max_objects
        self.min_confidence = min_confidence
        self.tracks = []
        self.object_count = 0

    def update(self, detections):
        current_detections = []
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            if conf > self.min_confidence:
                w = x2 - x1
                h = y2 - y1
                center = (int(x1 + w/2), int(y1 + h/2))
                bbox = (int(x1), int(y1), int(w), int(h))
                current_detections.append((center, bbox, int(cls)))

        new_tracks = []
        for detection in current_detections:
            center, bbox, cls = detection
            matched = False
            for track in self.tracks:
                if len(track['positions']) > 0:
                    last_pos = track['positions'][-1]
                    distance = np.sqrt((center[0] - last_pos[0])**2 +
                                     (center[1] - last_pos[1])**2)
                    if distance < 50 and track['class'] == cls:
                        track['positions'].append(center)
                        track['bbox'] = bbox
                        if len(track['positions']) > 20:
                            track['positions'].pop(0)
                        new_tracks.append(track)
                        matched = True
                        break

            if not matched:
                self.object_count += 1
                new_tracks.append({
                    'id': self.object_count,
                    'positions': deque([center], maxlen=20),
                    'bbox': bbox,
                    'class': cls
                })

        self.tracks = new_tracks
        return len(self.tracks)

def process_video(video_path, output_path=None):
    model = YOLO('yolov8n.pt')

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file")
        return

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    tracker = ObjectTracker()
    frame_count = 0
    class_names = model.names

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        results = model(frame)
        detections = results[0].boxes.data.cpu().numpy()
        object_count = tracker.update(detections)

        for track in tracker.tracks:
            x, y, w, h = track['bbox']
            class_name = class_names[track['class']]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track['id']} ({class_name})", (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            for i in range(1, len(track['positions'])):
                if track['positions'][i-1] is None or track['positions'][i] is None:
                    continue
                cv2.line(frame, track['positions'][i-1],
                        track['positions'][i], (0, 0, 255), 2)

        cv2.putText(frame, f"Objects: {object_count}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.putText(frame, f"Frame: {frame_count}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        if output_path:
            out.write(frame)

    cap.release()
    if output_path:
        out.release()

    print(f"Total frames processed: {frame_count}")
    if output_path:
        files.download(output_path)
# Upload video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
output_path = "output_video.mp4"

# Process the video
process_video(video_path, output_path)

Saving istockphoto-682889488-640_adpp_is.mp4 to istockphoto-682889488-640_adpp_is (3).mp4

0: 384x640 3 persons, 10.1ms
Speed: 2.2ms preprocess, 10.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 6.7ms
Speed: 2.0ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 6.6ms
Speed: 2.9ms preprocess, 6.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.4ms
Speed: 2.8ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.4ms
Speed: 2.7ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 suitcase, 6.7ms
Speed: 2.4ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 6.4ms
Speed: 1.8ms preprocess, 6.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>